### Data_anomalies_2NF_Exercise

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql

In [2]:
## Load the SoftDevEmployees database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///SoftDevEmployees.db

'Connected: @SoftDevEmployees.db'

In [ ]:
%%sql

DROP TABLE IF EXISTS Employees_2NF;
DROP TABLE IF EXISTS Titles_2NF;
DROP TABLE IF EXISTS Roles_2NF;
DROP TABLE IF EXISTS Departments_2NF;
DROP TABLE IF EXISTS Employee_Department_2NF;
DROP TABLE IF EXISTS Employee_Role_2NF;

CREATE TABLE Titles_2NF (
    TitleID INTEGER NOT NULL,
    Title VARCHAR,
    PRIMARY KEY(TitleID AUTOINCREMENT)
);

CREATE TABLE Roles_2NF (
    RoleID INTEGER NOT NULL,
    Role VARCHAR,
    PRIMARY KEY(RoleID AUTOINCREMENT)
);

CREATE TABLE  Departments_2NF (
    DepartmentID INTEGER NOT NULL,
    Department VARCHAR,
    PRIMARY KEY(DepartmentID AUTOINCREMENT)
);

CREATE TABLE Employees_2NF (
    EmployeeID INTEGER NOT NULL,
    Name VARCHAR, 
    Surname VARCHAR,
    Salary REAL,
    OccupationBand VARCHAR,
    TitleID INTEGER,
    FOREIGN KEY(TitleID) REFERENCES Titles_2NF (TitleID), 
    PRIMARY KEY(EmployeeID AUTOINCREMENT)
);


CREATE TABLE Employee_Role_2NF(
    EmployeeID INTEGER NOT NULL,
    RoleID INTEGER NOT NULL,
    FOREIGN KEY (EmployeeID) REFERENCES Employees_2NF (EmployeeID),
    FOREIGN KEY (RoleID) REFERENCES Roles_2NF (RoleID),
    PRIMARY KEY(EmployeeID, RoleID)
);

CREATE TABLE Employee_Department_2NF(
    EmployeeID INTEGER NOT NULL,
    DepartmentID INTEGER NOT NULL,
    FOREIGN KEY (EmployeeID) REFERENCES Employees_2NF (EmployeeID),
    FOREIGN KEY (DepartmentID) REFERENCES Departments_2NF (DepartmentID),
    PRIMARY KEY(EmployeeID, DepartmentID)
);

#### Populate the `Titles_2NF, Roles_2NF`, and `Departments_2NF` tables

In [ ]:
%%sql
DELETE FROM Titles_2NF;
DELETE FROM Roles_2NF;
DELETE FROM Departments_2NF;

INSERT INTO Titles_2NF (Title)
SELECT 
    DISTINCT Title 
FROM Employees_1NF 
WHERE Title <> '';

INSERT INTO Roles_2NF (Role) 
SELECT 
    DISTINCT Role
FROM Employees_1NF
WHERE Role <>'';

INSERT INTO Departments_2NF (Department)
SELECT
    DISTINCT Department
FROM Employees_1NF
WHERE Department <>'';

####  Populate the `Employees_2NF` table

In [ ]:
%%sql
DELETE FROM Employees_2NF;

INSERT INTO Employees_2NF (Name, Surname, Salary, OccupationBand, TitleID)
SELECT DISTINCT
    EMP.Name,
    EMP.Surname,
    EMP.Salary,
    EMP.OccupationBand,
    T.TitleID
FROM 
    Employees_1NF AS EMP
JOIN 
    Titles_2NF AS T 
    ON T.Title = EMP.Title;

####  Populate the `Employee_Department_2NF` and `Employee_Role_2NF` tables

In [ ]:
%%sql
DELETE FROM Employee_Department_2NF;
DELETE FROM Employee_Role_2NF;

INSERT INTO Employee_Department_2NF (EmployeeID,DepartmentID)
SELECT DISTINCT
    EMP2.EmployeeID,
    DPT.DepartmentID
FROM 
    Employees_1NF AS EMP1
JOIN 
    Employees_2NF AS EMP2 
    ON EMP1.Name = EMP2.Name AND EMP1.Surname = EMP2.Surname
JOIN 
    Departments_2NF AS DPT 
    ON EMP1.Department = DPT.Department;
    

INSERT INTO Employee_Role_2NF (EmployeeID,RoleID)
SELECT DISTINCT
    EMP2.EmployeeID,
    R.RoleID
FROM 
    Employees_1NF AS EMP1
JOIN 
    Employees_2NF AS EMP2 
    ON EMP1.Name = EMP2.Name AND EMP1.Surname = EMP2.Surname
JOIN 
    Roles_2NF AS R 
    ON EMP1.Role = R.Role